In [1]:
import os

In [2]:
%pwd

'/Users/anushachebolu/Documents/MLOps/9-data-science-project-with-deployment/research'

In [3]:
os.chdir("../")
%pwd

'/Users/anushachebolu/Documents/MLOps/9-data-science-project-with-deployment'

In [7]:
!pip install data dataclasses

### Update the entity

In [19]:
from dataclasses import dataclass  # Import the dataclass decorator for automatically generating special methods (e.g., __init__)
from pathlib import Path         # Import Path to handle filesystem paths in an OS-independent way

@dataclass  # The @dataclass decorator simplifies class creation by auto-generating methods like __init__ and __repr__
class DataIngestionConfig:
    # Root directory where all data ingestion artifacts will be stored
    root_Dir: Path

    # URL from which the data will be downloaded
    source_URL: str

    # Local file path where the downloaded data file will be saved
    local_data_file: Path

    # Directory where the downloaded file will be extracted
    unzip_dir: Path


### Update the configuration manager in src config

In [22]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config


### Update the Components

In [24]:
## component-Data Ingestion
import os
import urllib.request as request
from src.datascience import logger
import zipfile

class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e